In [ ]:
import requests

api_key = 'CIVEY6DJ1KKAUPZK'
symbol = 'AAPL'

url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}'
response = requests.get(url)
data = response.csv()

print(data)